In [1]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from datetime import datetime
from scipy.stats.stats import pearsonr

# Data Understanding

In [2]:
# Loading dataset.

custom_date_parser = lambda x: datetime.strptime(x, "%d/%m/%y %H:%M")

df = pd.read_csv('customer_supermarket.csv', sep='\t', decimal=',',
                 parse_dates=['BasketDate'], date_parser=custom_date_parser, index_col=0)

df['CustomerID'] = df['CustomerID'].astype(float)

In [3]:
df.dtypes

BasketID                   object
BasketDate         datetime64[ns]
Sale                      float64
CustomerID                float64
CustomerCountry            object
ProdID                     object
ProdDescr                  object
Qta                         int64
dtype: object

# Data Quality 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471910 entries, 0 to 541909
Data columns (total 8 columns):
BasketID           471910 non-null object
BasketDate         471910 non-null datetime64[ns]
Sale               471910 non-null float64
CustomerID         406830 non-null float64
CustomerCountry    471910 non-null object
ProdID             471910 non-null object
ProdDescr          471157 non-null object
Qta                471910 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.4+ MB


In [5]:
df.isnull().sum()

BasketID               0
BasketDate             0
Sale                   0
CustomerID         65080
CustomerCountry        0
ProdID                 0
ProdDescr            753
Qta                    0
dtype: int64

In [6]:
df.describe()

,Sale,CustomerID,Qta
count,471910.000000,406830.000000,471910.000000
mean,4.030945,15287.684160,10.716533
std,83.769380,1713.603074,231.355136
min,-11062.060000,12346.000000,-80995.000000
25%,1.250000,13953.000000,1.000000
50%,2.080000,15152.000000,4.000000
75%,3.750000,16791.000000,12.000000
max,38970.000000,18287.000000,80995.000000


In [7]:
df[df['Sale'] < 0]

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
299983,A563186,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,B,Adjust bad debt,1
299984,A563187,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,B,Adjust bad debt,1


In [8]:
df[df['Qta'] < 0]

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
141,C536379,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,D,Discount,-1
154,C536383,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,35004C,SET OF 3 COLOURED FLYING DUCKS,-1
235,C536391,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,22556,PLASTERS IN TIN CIRCUS PARADE,-12
236,C536391,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,21984,PACK OF 12 PINK PAISLEY TISSUES,-24
237,C536391,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24
...,...,...,...,...,...,...,...,...
540449,C581490,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11
541541,C581499,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,M,Manual,-1
541715,C581568,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,21258,VICTORIAN SEWING BOX LARGE,-5
541716,C581569,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,84978,HANGING HEART JAR T-LIGHT HOLDER,-1


In [11]:
df['CustomerCountry'].value_counts()

United Kingdom          426261
Germany                   9495
France                    8525
EIRE                      7824
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               1932
Portugal                  1495
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Unspecified                340
USA                        291
Israel                     269
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Hong Kong                  138
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

# Data Cleaning 

In [12]:
aborted = "A"
canceled = "C"
df['Status'] = df['BasketID'].apply(lambda x : 'Canceled' if x.startswith(canceled) 
                                    else ('Aborted' if x.startswith(aborted) else 'Shipped'))

df['BasketID'] = df['BasketID'].apply(lambda x : x[1:] if x.startswith(canceled) or x.startswith(aborted) else x)

In [13]:
df['Qta'] = np.where(df['Status'] == 'Canceled', abs(df['Qta']), df['Qta'])
df['Sale'] = np.where(df['Status'] == 'Aborted', abs(df['Sale']), df['Sale'])
df['BasketID'] = df['BasketID'].astype(int)

In [14]:
df.dtypes

BasketID                    int64
BasketDate         datetime64[ns]
Sale                      float64
CustomerID                float64
CustomerCountry            object
ProdID                     object
ProdDescr                  object
Qta                         int64
Status                     object
dtype: object

In [20]:
df.corr() # Method: {‘pearson’, ‘kendall’, ‘spearman’}

,BasketID,Sale,CustomerID,Qta
BasketID,1.000000,-0.003183,0.001425,-0.000279
Sale,-0.003183,1.000000,-0.004560,-0.001381
CustomerID,0.001425,-0.004560,1.000000,-0.005101
Qta,-0.000279,-0.001381,-0.005101,1.000000
